# (MIDI) Prédiction et génération de fichier midi

Lien du sujet : https://github.com/neuronalX/Reservoir-Jupyter
Apprendre le reservoir computing : https://github.com/neuronalX/Reservoir-Jupyter
Code d'exemple reservoir computing : https://github.com/neuronalX/FunkyReservoir

Import des modules

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from ipywidgets import *
from IPython.display import *

Classe pour générer le réseau :

In [6]:
class Network(object):

    def __init__(self, trainLen=2000, testLen=2000, initLen=100, data="MackeyGlass_t17.txt") :
        self.initLen = initLen
        self.trainLen = trainLen
        self.testLen = testLen
        self.data = np.loadtxt(data)
        self.inSize = self.outSize = 1 #Dimension de l'entrée et de la sortie
        self.resSize = 300 #Taille du réservoir (prédiction)
        #self.resSize = 1000 #Taille du réservoir (géneration)
        self.a = 0.3 #Taux de fuite alpha 
        self.spectral_radius = 1.25 #Rayon spectral
        self.input_scaling = 1. #Echelle de l'input
        self.reg =  1e-8 #None #Coefficient de régularisation - si None,
        #on utilisera le pseudo-inverse plutôt que la régression de crête

        self.mode = 'prediction'
        #self.mode = 'generative'

        #Modification de la première valeur pseudo-aléatoire : les performances du réservoir
        #devraient être dans la moyenne après 20 instances aléatoires (pour les mêmes paramètres)
        seed = self.set_seed()
        
        #Poids
        self.Win = (np.random.rand(self.resSize,1+self.inSize)-0.5) * self.input_scaling
        self.W = np.random.rand(self.resSize,self.resSize)-0.5 

        #Matrices
        #Matrice des états collectés (1,u,x) au cours du temps
        self.X = np.zeros((1+self.inSize+self.resSize,self.trainLen-self.initLen))
        #Matrice de sortie cible
        self.Ytarget = self.data[None,self.initLen+1:self.trainLen+1]

        #Vecteur des états du réservoir
        self.x = np.zeros((self.resSize,1))  
        
    def set_seed(self):
        """Making the seed (for random values) variable if None"""

        if seed is None:
            import time
            seed = int((time.time()*10**6) % 4294967295)
        try:
            np.random.seed(seed)
            print("Seed used for random values:", seed)
        except:
            print("!!! WARNING !!!: Seed was not set correctly.")
        return seed

nw = Network(trainLen=2000, testLen=2000, initLen=100, data="MackeyGlass_t17.txt")

Seed used for random values: 166609793


Récupération des données